In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline, AutoModelForSeq2SeqLM, TextStreamer
from pydantic import BaseModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
import torch
# import intel_extension_for_pytorch as ipex
import ast
from bs4 import BeautifulSoup
import requests
import json
from tqdm import tqdm
import re


class JobFormat(BaseModel):
    title: str
    company: str
    location: str
    application_link: str


def generate_tag(pipe,resume):

    prompt = f""" <s> [INST]

    You're an LLM who's role is to read the content of the resume, and provide job titles related to their resume that you can generate from that. Eg. [Machine Learning Engineer, Software development Engineer, Backend Engineer] which can be used as filters for job-hunting. Your response should strictly follow the syntax of a python list.


    Resume_content: {resume} [/INST]

    """



    # print("Generating response...")
    output = pipe(prompt)[0]["generated_text"]

    response_index = output.find("/INST]")

    response = output[response_index+6:]

    pattern = r'(?<=\[)(.*?)(?=\])'

    # Use regex to find the list
    matches = re.search(pattern, response)
    
    if matches:
        output = matches.group(0)
        print(output)

    response = f"[{output}]"

    return response


def validate_jobs(pipe,resume,jobs_json,preferences):


    output_format = JobFormat.schema()

#     output_format= {
        
#     "type": "object",
#     "properties": {
#         "jobs": {
#             "type": "array",
#             "items": {
#                 "type": "object",
#                 "properties": {
#                     "title": {"type": "string"},
#                     "company": {"type": "string"},
#                     "location": {"type": "string"},
#                     "application_link": {"type": "string"}
#                 },
#                 "required": ["title", "company", "location", "application_link"],
#                 "additionalProperties": False
#             }
#         }
#     }
# }

    
    prompt = f"""<s>[INST]

    You're an LLM who's role is to recieve a list of available jobs, a list of preferences and a resume as input, and your output will be a JSON RESPONSE of the top 10 most relevant jobs among the given jobs and should contain absolutely no texts other than that, based on the contents of the resume and user's preferences. The output should strictly follow the same format as the input json.

    Resume_content: {resume}, jobs: {jobs_json}, preferences: {preferences} [/INST]

"""

    parser = JsonSchemaParser(JobFormat.model_json_schema())
    prefix_function = build_transformers_prefix_allowed_tokens_fn(pipe.tokenizer, parser)

    # Call the pipeline with the prefix function
    output = pipe(prompt, prefix_allowed_tokens_fn=prefix_function)

    return output
    

def get_linkedin(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    job_cards = soup.find_all("div", class_="base-search-card__info")

    all_job_details = []

    # Iterate over each job card
    for card in job_cards:
        # Find the job details
        job_title = card.find("h3", class_="base-search-card__title").text.strip()
        print(f"job_title: {job_title}")
        try:
            company_name = card.find("a", class_="hidden-nested-link").text.strip()
        except:
            company_name="Unavailable"
        location = card.find("span", class_="job-search-card__location").text.strip()
        salary_info = card.find("span", class_="job-search-card__salary-info")

        if salary_info:
            salary_info = salary_info.text.strip()
        else:
            salary_info="Salary Info not available"
        try:
            apply_link = card.find("a", class_="hidden-nested-link")["href"]
        except:
            apply_link="unavailable"
        

        # Create a dictionary to store the data for this job
        job_details = {
            "job_title": job_title,
            "company_name": company_name,
            "location": location,
            "salary_info": salary_info,
            "Link":apply_link
        }

        # Append the job details to the list
        all_job_details.append(job_details)

    return all_job_details

def create_search_querries(tags:str):

    tags = tags.replace('\n','')

    tags = tags.strip()
    
    tags_list = ast.literal_eval(tags)


    jobs = []

    for tag in tags_list[:5]:

        linkedin_keywords = '%20'+tag.replace(' ','%20')


        linkedin_query = f"https://www.linkedin.com/jobs/search?keywords={linkedin_keywords}&location=US&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum={0}"

        jobs.append(get_linkedin(linkedin_query))

    internshala_keywords = ''
    for tag in tags_list:
        internshala_keywords+=tag.replace(' ','-')+','

    internshala_keywords= internshala_keywords[:-1]

    internshala_search_query = f"https://internshala.com/internships/{internshala_keywords}-internship"


    return jobs


def get_internshala(url):

    response = requests.get(url)

    soup = BeautifulSoup(response.content, "html.parser")

    internship_containers = soup.find_all("div", class_="internship_meta")

    all_internship_details = []

    for container in internship_containers:
        # Find the internship details
        internship_name = container.find("h3", class_="heading_4_5 profile").text.strip()
        company_name = container.find("a", class_="link_display_like_text view_detail_button").text.strip()
        location = container.find("a", class_="location_link view_detail_button").text.strip()
        start_date = container.find("div", class_="item_body").text.strip()
        duration = container.find_all("div", class_="item_body")[1].text.strip()
        stipend = container.find("span", class_="stipend").text.strip()
        applying_link = f"""https://internshala.com{soup.find("a", class_="view_detail_button")['href']}"""

        # Create a dictionary to store the data for this internship
        internship_details = {
            "Internship Name": internship_name,
            "Company Name": company_name,
            "Location": location,
            "Start Date": start_date,
            "Duration": duration,
            "Stipend": stipend,
            "Applying Link": applying_link
        }

        # Append the internship details to the list
        all_internship_details.append(internship_details)

    return all_internship_details


def get_results(resume_content,preferences):


    tag_generator_path = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ"

    # tag_generator_path = "HuggingFaceH4/mistral-7b-grok"

    tag_generator = AutoModelForCausalLM.from_pretrained(
        tag_generator_path,
        low_cpu_mem_usage=True,
        device_map="auto"
    )

    # tag_generator = ipex.optimize(tag_generator,dtype=torch.float16)

    tokenizer = AutoTokenizer.from_pretrained(tag_generator_path)

    tag_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_params = {
        "do_sample": True,
        "temperature": 0.1,
        "top_p": 0.95,
        "top_k": 40,
        "max_new_tokens": 2048,
        "repetition_penalty": 1.1
    }

    tag_pipe = pipeline(
        "text-generation",
        model=tag_generator,
        tokenizer=tokenizer,
        streamer=tag_streamer,
        device_map="auto",
        **generation_params
    )

    

    # validator_path = "mistralai/Mixtral-8x7B-Instruct-v0.1"
        
    # validator = AutoModelForCausalLM.from_pretrained(
    #     validator_path,
    #     low_cpu_mem_usage=True,
    #     device_map="auto"
    # )
    # # validator = ipex.optimize(validator, dtype=torch.float16)
    # tokenizer = AutoTokenizer.from_pretrained(validator_path)
    
    # val_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    
    # generation_params = {
    #     "do_sample": True,
    #     "temperature": 0.7,
    #     "top_p": 0.95,
    #     "top_k": 40,
    #     "max_new_tokens": 1000,
    #     "repetition_penalty": 1.1
    # }
    
    # val_pipe = pipeline(
    #     "text-generation",
    #     model=validator,
    #     tokenizer=tokenizer,
    #     streamer=val_streamer,
    #     **generation_params
    # )


    jobs = (create_search_querries(generate_tag(tag_pipe,resume_content)))

    print("jobs created")

    print(jobs)
    
    jobs_list = []

    for i in jobs:
        if len(i)>1:
            for j in i:
                jobs_list.append(j)

    split_ratio = len(jobs_list)//6

    job_splits = []

    for i in range(0, len(jobs_list), split_ratio):
        job_splits.append(jobs_list[i:i+split_ratio])

    print(len(job_splits))

    validated = ""

    for split in tqdm(job_splits):

        validated += validate_jobs(pipe=tag_pipe,resume=resume_content,jobs_json=split,preferences=preferences)

    print(validated)

    

    return validated

/home/tejasram/miniconda3/envs/langchain_pdf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(get_results(resume_content,preferences))

/home/tejasram/miniconda3/envs/langchain_pdf/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/tejasram/miniconda3/envs/langchain_pdf/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-03-11 13:31:19,129 - accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["Machine Learning Researcher", "Generative Adversarial Networks Optimizer", "Data Analyst (Python, Numpy, Pandas, Matplotlib, Seaborn, Hadoop)", "Computer Vision Engineer", "Natural Language Processing Engineer", "Deep Learning Engineer (PyTorch, TensorFlow, Scikit-Learn)", "Full Stack Developer (Django, React.js)" ]
"Machine Learning Researcher", "Generative Adversarial Networks Optimizer", "Data Analyst (Python, Numpy, Pandas, Matplotlib, Seaborn, Hadoop)", "Computer Vision Engineer", "Natural Language Processing Engineer", "Deep Learning Engineer (PyTorch, TensorFlow, Scikit-Learn)", "Full Stack Developer (Django, React.js)" 
job_title: Machine Learning Research Scientist
job_title: AI Researcher
job_title: Machine Learning Consultant
job_title: AI Scientist
job_title: AI Researcher, Core ML
job_title: AI Research Scientist
job_title: Artificial Intelligence Researcher
job_title: Machine Learning Researcher
job_title: Machine Learning Co-op
job_title: Machine Learning Scientist 4 -

  0%|          | 0/7 [00:00<?, ?it/s]/tmp/ipykernel_8776/757431481.py:59: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  output_format = JobFormat.schema()
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 0/7 [00:03<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 438.00 MiB. GPU 0 has a total capacity of 5.80 GiB of which 213.69 MiB is free. Including non-PyTorch memory, this process has 5.58 GiB memory in use. Of the allocated memory 5.05 GiB is allocated by PyTorch, and 396.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [69]:
torch.cuda.empty_cache()

In [3]:
resume_content = """

Summary: Summary: Motivated undergraduate in Artificial Intelligence and Data Science, with aspirations to excel as a Machine 
Learning Engineer and Software Developer. Possessing expertise in Computer Vision, Generative AI, and 
Natural Language Processing, I am actively pursuing internships to apply this knowledge in real-world 
settings and make meaningful contributions to the domains of AI and software development.

Languages:
Python
JavaScript
HTML
SQL

Data Analytics Tools:
Numpy
Pandas
Matplotlib
Seaborn
Hadoop


Domain Skills:
Computer Vision
NLP
Image Processing
GANs


Frameworks:
Pytorch
Tensorflow
Scikit-Learn
Django
React.js


Experience: Machine Learning Research Intern NIT Trichy 

Conducted a research-based internship focused on optimizing Generative Adversarial Networks 
(GANs) to enhance data augmentation efficiency, resulting in reduced training time and improved 
accuracy, culminating in the publication of a groundbreaking research paper.
• Collaborated with fellow interns to analyze potential obstacles in training GANs for data augmentation 
purposes, and successfully devised and implemented algorithms to mitigate and overcome these 
challenges.
• Developed a Machine Learning Pipeline Python using PyTorch and scikit-learn to detect image 
forgeries, successfully identifying and highlighting suspicious areas in images through computer vision 
techniques and neural networks.


Projects:

Crime Hotspot Map
A Machine Learning project on 
predicting the likeliness of user 
being a victim of crime based on 
their current location using the K-
Nearest Neighbour Regressor 
algorithm, integrated with Leaflet 
API and Django Framework.


Brain Tumour Detection on MRIs
Designed and deployed a web-
based diagnostic system using 
Convolutional Neural Networks 
(CNNs) and TensorFlow, enabling 
accurate detection and 
classification of brain tumours in 
MRI scan images, with the
application built on the Django 
framework.


Image to Prompt
A CNN and LSTM based model
developed using Pytorch to 
generate descriptive prompts from 
images, inverting the Stable 
Diffusion Image generation 
approach.

"""

preferences = "I would prefer a development role over a research role"